In [4]:
# Paso 1: Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Paso 2: Importar librerías necesarias
import pandas as pd
from IPython.display import display, HTML

# Paso 3: Cargar archivo CSV
archivo = '/content/drive/MyDrive/CADI_Personal/Proyecto/Datos_Roster_V2.csv'
df = pd.read_csv(archivo)

# Limpiar espacios en nombres de columnas
df.columns = df.columns.str.strip()

# Paso 4: Clasificar tipo de UF
def clasificar_tipo_uf(uf):
    if isinstance(uf, str):
        if uf.endswith("S"):
            return "Semana Tec"
        elif uf.endswith("B"):
            return "Bloque"
        elif uf.endswith("C"):
            return "Concentración"
        else:
            return "Materia"
    return "Desconocido"

df['Tipo de UF'] = df['UF'].apply(clasificar_tipo_uf)

# Paso 5: Detectar coordinadores por Carga Co.
if 'Carga Co.' in df.columns:
    coordinadores = df[df['Carga Co.'].notnull()][['UF', 'Grupo', 'Profesor', 'Correo']].copy()
    coordinadores = coordinadores.rename(columns={
        'Profesor': 'Coordinador',
        'Correo': 'Correo Coordinador'
    })
    df = df.merge(coordinadores, on=['UF', 'Grupo'], how='left')
else:
    print("Advertencia: No se encontró la columna 'Carga Co.' en el archivo CSV.")

# Paso 6: Input del número de nómina
nomina = input("Ingresa tu número de nómina (ej. L01234567): ").strip()

# Paso 7: Filtrar asignaciones del profesor
datos_profesor = df[df['Nómina'] == nomina].copy()

# Paso 8: Mostrar tabla si hay resultados
if datos_profesor.empty:
    print("No se encontraron asignaciones para esa nómina.")
else:
    # Asegurar que 'Carga Co.' y 'UDCs' sean numéricos
    datos_profesor['Carga Co.'] = pd.to_numeric(datos_profesor.get('Carga Co.', 0), errors='coerce').fillna(0)
    datos_profesor['UDCs'] = pd.to_numeric(datos_profesor.get('UDCs', 0), errors='coerce').fillna(0)

    # Calcular y redondear totales
    total_carga_co = round(datos_profesor['Carga Co.'].sum(), 2)
    total_udcs = round(datos_profesor['UDCs'].sum(), 2)
    udcs_totales = round(total_udcs + total_carga_co, 2)

    # Agregar columna de coordinador visible si aplica
    def mostrar_coordinador(row):
        if row['Tipo de UF'] in ['Bloque', 'Concentración']:
            return f"{row.get('Coordinador', '')} ({row.get('Correo Coordinador', '')})"
        return ""

    datos_profesor['Coordinador de Bloque'] = datos_profesor.apply(mostrar_coordinador, axis=1)

    # Convertir grupo a texto sin decimales
    datos_profesor['Grupo'] = datos_profesor['Grupo'].fillna('').apply(
        lambda x: str(int(x)) if isinstance(x, float) else str(x)
    )

    # Columnas que se mostrarán
    columnas = [
        'UF', 'Grupo', 'Nombre de UF', 'Inglés', 'Tipo de UF',
        '% de Resp', 'UDCs', 'Periodo', 'Horario', 'Coordinador de Bloque'
    ]
    resultado = datos_profesor[columnas]

    # Estilo visual + clase CSS para centrado completo
    estilo = """
    <style>
      table.tabla-centro {
        border-collapse: collapse;
        width: 100%;
        font-family: Verdana, Segoe UI, sans-serif;
        margin-bottom: 20px;
      }
      .tabla-centro th {
        background-color: #003366;
        color: white;
        padding: 10px;
        font-size: 15px;
        text-align: center;
        font-family: 'Segoe UI', Verdana, sans-serif;
      }
      .tabla-centro td {
        background-color: #f9f9f9;
        padding: 10px;
        border-bottom: 1px solid #ccc;
        font-size: 14px;
        text-align: center;
      }
      .tabla-centro tr:hover td {
        background-color: #e6f2ff;
      }
      .resumen-container {
        display: flex;
        gap: 20px;
        margin-top: 20px;
        font-family: 'Segoe UI', Verdana, sans-serif;
      }
      .card {
        background-color: #f0f8ff;
        border-left: 6px solid #003366;
        padding: 15px 20px;
        flex: 1;
        border-radius: 6px;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        text-align: center;
      }
      .card strong {
        display: block;
        font-size: 16px;
        color: #003366;
        margin-bottom: 5px;
      }
      .card span {
        font-size: 22px;
        font-weight: bold;
        color: #000;
      }
    </style>
    """

    # Generar tabla con clase personalizada para centrado
    tabla_html = resultado.to_html(index=False, escape=False, na_rep='', classes='tabla-centro')

    resumen_html = f"""
    <div class="resumen-container">
      <div class="card">
        <strong>Total UDCs Docente</strong>
        <span>{total_udcs}</span>
      </div>
      <div class="card">
        <strong>Total UDCs Coordinación</strong>
        <span>{total_carga_co}</span>
      </div>
      <div class="card">
        <strong>UDCs Totales</strong>
        <span>{udcs_totales}</span>
      </div>
    </div>
    """

    # Mostrar en pantalla
    display(HTML(estilo + tabla_html + resumen_html))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ingresa tu número de nómina (ej. L01234567): L03557709


UF,Grupo,Nombre de UF,Inglés,Tipo de UF,% de Resp,UDCs,Periodo,Horario,Coordinador de Bloque
M2005B,303,Design of Thermofluidic Systems,SI,Bloque,55%,2.2,PMT3,LuMaJuVi 15:00-19:00,Sergio Maldonado Villanueva (s.maldonado@tec.mx)
M2005B,305,Diseño de sistemas termofluídicos,,Bloque,55%,2.2,PMT3,LuMaJuVi 11:00-15:00,Sergio Maldonado Villanueva (s.maldonado@tec.mx)
M2036,101,Fundamentals of Fluid Mechanic,SI,Materia,100%,1.0,PMT1,LuJu 11:00-13:00,
M2037,204,Análisis de los procesos de transformación energética,,Materia,100%,1.0,PMT2,LuJu 15:00-17:00,
M3043,603,Estancia de investigación,,Materia,100%,0.0,PMT6,,
M3043,604,Estancia de investigación,,Materia,40%,0.5,PMT6,,
